In [1]:
from flask import Flask, json, request
from urllib.parse import urlparse
import requests
import ast
from Client_Model import *

In [ ]:
api = Flask(__name__)
from flask import session, url_for, redirect

@api.route('/register', methods=['GET'])
def register():
    
    o = urlparse(request.base_url)
    PARAMS = {'url':o.hostname, 'port':o.port}
    r = requests.get(url = 'http://127.0.0.1:10011/register', params = PARAMS)
    
    #print(r, r.status_code, r.reason, r.text)
    if r.status_code == 200:
        return "Status OK sent !"
    
    return "Failed Status"

@api.route('/active_test', methods=['GET'])
def active_test():
    ack = {'client_ack': '1'}
    return str(ack)

@api.route('/downloadmodel', methods=['POST'])
def get_agg_model():
    if request.method == 'POST':
        file = request.files['model'].read()
        fname = request.files['json'].read()

        fname = ast.literal_eval(fname.decode("utf-8"))
        fname = fname['fname']
        #print(fname)

        wfile = open(fname, 'wb+')
        wfile.write(file)

        return "Model received!"
    else:
        return "No file received!"
    
@api.route('/sendmodel', methods=['GET'])
def send_model():
    client_train()
    print("Training over")
    file = open("agg_model.npy", 'rb')

    o = urlparse(request.base_url)
    data = {'fname':'agg_model.npy', 'url':o.hostname, 'port':o.port}
    files = {
        'json': ('json_data', json.dumps(data), 'application/json'),
        'model': ('agg_model.npy', file, 'application/octet-stream')
    }

    req = requests.post(url='http://127.0.0.1:10011/client_model', files=files)
    return "Model sent !"
    
if __name__ == '__main__':
    api.secret_key = '2458867997' ##set to random
    api.config['SESSION_TYPE'] = 'filesystem'
    api.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Jul/2022 15:36:11] "GET /active_test HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2022 15:36:12] "POST /downloadmodel HTTP/1.1" 200 -


Processing data...
10000 test samples
Agg weights exists...
Loading weights...
469/469 [==============================] - 39s 83ms/step - loss: 0.0844 - accuracy: 0.9746 - val_loss: 0.0428 - val_accuracy: 0.9866


C:\Users\pk15425\Anaconda3\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
127.0.0.1 - - [13/Jul/2022 15:37:28] "GET /sendmodel HTTP/1.1" 200 -


Test loss: 0.04280697926878929
Test accuracy: 0.9865999817848206
Local model update written to local storage!
Training over


127.0.0.1 - - [13/Jul/2022 15:38:10] "GET /active_test HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2022 15:38:10] "POST /downloadmodel HTTP/1.1" 200 -
